In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option("display.max_rows", 2000)

webpage_response =requests.get("https://www.scotchwhiskyauctions.com/auctions/181-the-137th-auction/")

webpage = webpage_response.content
soup = BeautifulSoup(webpage, "html.parser")



In [3]:
print(soup.find_all(attrs={"class": "sold"}))
#print(soup.find_all("p"))

[<p class="sold">Sold for £90</p>, <p class="sold">Sold for £90</p>, <p class="sold">Sold for £65</p>, <p class="sold">Sold for £230</p>, <p class="sold">Sold for £170</p>, <p class="sold">Sold for £60</p>, <p class="sold">Sold for £85</p>, <p class="sold">Sold for £80</p>, <p class="sold">Sold for £75</p>, <p class="sold">Sold for £80</p>, <p class="sold">Sold for £80</p>, <p class="sold">Sold for £260</p>, <p class="sold">Sold for £25</p>, <p class="sold">Sold for £60</p>, <p class="sold">Sold for £75</p>, <p class="sold">Sold for £75</p>, <p class="sold">Sold for £80</p>, <p class="sold">Sold for £75</p>, <p class="sold">Sold for £30</p>]


In [4]:
items = soup.find_all(attrs={"class": "lot"})
print(items)
   

[<a class="lot" href="auctions/181-the-137th-auction/609239-glenfarclas-8-year-old-for-whisky-over-ice-festival-2022-/"><div class="hasnotbid" id="status_609239"></div><div class="aucimgw"><div class="aucimg hasretina" rel="orig224" style="background-image: url('https://d3suvtcq00dftb.cloudfront.net/images/32aacb27566a470d8974d1c1e4744550-med.jpg');"></div></div><h4>Glenfarclas 8 Year Old For Whisky Over Ice Festival 2022 </h4><h6>Lot no 137-07769</h6><div id="info_609239"><p class="bndone">This lot is no longer available for sale.<br/>(Price £39)</p></div></a>, <a class="lot" href="auctions/181-the-137th-auction/604804-on-a-saw-mill-10-year-old-islay-blended-malt/"><div class="hasnotbid" id="status_604804"></div><div class="aucimgw"><div class="aucimg hasretina" rel="orig224" style="background-image: url('https://d3suvtcq00dftb.cloudfront.net/images/716bf42ba0261256845efea455e23b55-med.jpg');"></div></div><h4>'On A Saw Mill' 10 Year Old Islay Blended Malt</h4><h6>Lot no 137-03432</h6>

In [5]:

sold_prices = soup.find_all(attrs={"class": "sold"})
not_sold_prices = soup.find_all(attrs={"class": "bndone"}) # not sold bottles


bottle_ids = soup.find_all("h6")                                          # THINK WE ARE GOING TO BE ABLE TO DELETE ALL THIS AS WE GETTING ALL THE INFO WE WANT ON EACH INDIVIDUAL BOTTLES PAGE. 
bottle_names = soup.find_all("h4")[3:]
all_bottles_prices = soup.find_all(class_= ["sold", "bndone"])

In [6]:
# Getting the links
links = []
for a in soup.find_all("a", href=True)[21:41]:
    links.append(a["href"])
print(links)

['auctions/181-the-137th-auction/609239-glenfarclas-8-year-old-for-whisky-over-ice-festival-2022-/', 'auctions/181-the-137th-auction/604804-on-a-saw-mill-10-year-old-islay-blended-malt/', 'auctions/181-the-137th-auction/603808-yer-aunt-fannys-cats-arsehole-whiskyblender-50cl/', 'auctions/181-the-137th-auction/596096-a-speyside-13-year-old-cadenheads/', 'auctions/181-the-137th-auction/607176-a-speyside-1989-32-year-old-thompson-bros/', 'auctions/181-the-137th-auction/606262-a-speyside-2003-18-year-old-cadenheads-warehouse-tasting/', 'auctions/181-the-137th-auction/605873-a-speyside-2010-11-year-old-thompson-bros-/', 'auctions/181-the-137th-auction/606480-a-speyside-distillery-18-year-old-cadenheads/', 'auctions/181-the-137th-auction/605619-a-speyside-distillery-2001-18-year-old-cadenheads-virtual-open-day-2020/', 'auctions/181-the-137th-auction/605214-a-speyside-distillery-2008-12-year-old-single-malts-of-scotland-parcel-no6/', 'auctions/181-the-137th-auction/594430-a-speyside-distiller

In [7]:
# Putting the links together as the individual bottles links miss the actual website link.
full_links = []
for link in links:
    full_links.append("http://scotchwhiskyauctions.com/" + link)
print(full_links)

['http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/609239-glenfarclas-8-year-old-for-whisky-over-ice-festival-2022-/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/604804-on-a-saw-mill-10-year-old-islay-blended-malt/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/603808-yer-aunt-fannys-cats-arsehole-whiskyblender-50cl/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/596096-a-speyside-13-year-old-cadenheads/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/607176-a-speyside-1989-32-year-old-thompson-bros/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/606262-a-speyside-2003-18-year-old-cadenheads-warehouse-tasting/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/605873-a-speyside-2010-11-year-old-thompson-bros-/', 'http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/606480-a-speyside-distillery-18-year-old-cadenheads/', 'http://scotchwhiskyauctions.com/a

In [8]:
pages_links = []
counter = 0
for url in full_links:

    getting_whiskey_info = requests.get(url)

    if getting_whiskey_info.status_code == 200:  # status code 200 is a response indicating the request was successful. 
        print(f"URL #{counter+1}: {url}")  # To see what weve requested so far successfully.
    else:
        print(f"Status code {getting_whiskey_info.status_code}: Skipping URL #{counter+1}")   # The reason its i+1 one is because we dont increment till the end of the loop, so the current iteration actually needs incrementing. 
        counter += 1
        continue  # This else statement is essentially saying if the request wasnt successful continue onto the next element. 

    bottle_content = getting_whiskey_info.content # The content i think
    
    individual_bottles_soup = BeautifulSoup(bottle_content, "html.parser")
    
    bottles_pars = individual_bottles_soup.find_all("p") #  storing all paragraph tags in a variable. 
    
    text = [p.text for p in bottles_pars] # extracting the text
    
    pages_links.append(text)
    
    counter += 1

URL #1: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/609239-glenfarclas-8-year-old-for-whisky-over-ice-festival-2022-/
URL #2: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/604804-on-a-saw-mill-10-year-old-islay-blended-malt/
URL #3: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/603808-yer-aunt-fannys-cats-arsehole-whiskyblender-50cl/
URL #4: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/596096-a-speyside-13-year-old-cadenheads/
URL #5: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/607176-a-speyside-1989-32-year-old-thompson-bros/
URL #6: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/606262-a-speyside-2003-18-year-old-cadenheads-warehouse-tasting/
URL #7: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/605873-a-speyside-2010-11-year-old-thompson-bros-/
URL #8: http://scotchwhiskyauctions.com/auctions/181-the-137th-auction/606480-a-speyside-distillery-18-year-old-cadenhe

In [15]:
print(pages_links[0])

['Lot number: 137-07769', 'Price: £39.', 'This lot is no longer available for sale.', "Still in family hands since it was founded, the Glenfarclas Distillery is one of Speyside's independent whisky producers. The Grant family has been running Glenfarclas since 1860, producing an individual Speyside malt that many whisky enthusiasts number as one of their favourites. The water source is melt water from the slopes of Ben Rinnes which runs through granite and heather before reaching Glenfarclas (meaning 'the valley of green grass'). With a long history of bottlings stretching back to the early 20th Century, both proprietary and independent bottlings are much sought-after.", 'This 8 Year Old was bottled for The Real Drinks Company for the Whisky Over Ice Festival 2022.', 'Bottled: 03.05.2022', 'Age: 8 Years Old', '48.8% ABV / 70cl', 'Bottle Number: One of only 600 bottles released', 'Please note buyers premium does not apply to these bottles.', 'Please examine all images of the lot, any pa

In [10]:
first_values = []
for bottle in pages_links[0]:
    print("info:", bottle)
    first_values.append(bottle)    # To give a clearer idea of what they are made up of. 

info: Lot number: 137-07769
info: Price: £39.
info: This lot is no longer available for sale.
info: Still in family hands since it was founded, the Glenfarclas Distillery is one of Speyside's independent whisky producers. The Grant family has been running Glenfarclas since 1860, producing an individual Speyside malt that many whisky enthusiasts number as one of their favourites. The water source is melt water from the slopes of Ben Rinnes which runs through granite and heather before reaching Glenfarclas (meaning 'the valley of green grass'). With a long history of bottlings stretching back to the early 20th Century, both proprietary and independent bottlings are much sought-after.
info: This 8 Year Old was bottled for The Real Drinks Company for the Whisky Over Ice Festival 2022.
info: Bottled: 03.05.2022
info: Age: 8 Years Old
info: 48.8% ABV / 70cl
info: Bottle Number: One of only 600 bottles released
info: Please note buyers premium does not apply to these bottles.
info: Please exa

In [13]:
#put_together = list(zip(bottle_ids, bottle_names, all_bottles_prices, full_links))
#print(put_together)